In [2]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python

In [3]:
!pip install langchain

In [5]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate

In [6]:
template = """Question: {question}

Answer: Let's work this out in a step by step way to be sure we have the right answer."""

prompt = PromptTemplate(template=template, input_variables=["question"])

In [7]:
!wget get "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf"

--2023-12-09 04:54:04--  http://get/
Resolving get (get)... failed: Name or service not known.
wget: unable to resolve host address ‘get’
--2023-12-09 04:54:04--  https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf
Resolving huggingface.co (huggingface.co)... 13.33.33.110, 13.33.33.20, 13.33.33.102, ...
Connecting to huggingface.co (huggingface.co)|13.33.33.110|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/8d/b1/8db1d1f73b4caa58e947ccbfe2fb27ac5e495c2ad8457ad299d15987aee3b520/eda2a15d532bea4ce6fc14d15c2b72638243396816252f73a94dceeb0429112f?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27llama-2-13b-chat.Q4_0.gguf%3B+filename%3D%22llama-2-13b-chat.Q4_0.gguf%22%3B&Expires=1702356844&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwMjM1Njg0NH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy84ZC9iMS84Z

In [8]:
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path="/content/llama-2-13b-chat.Q4_0.gguf",
    temperature=0.75,
    max_tokens=2000,
    top_p=1,
    callback_manager=callback_manager,
    verbose=True,
    n_ctx = 2048,
    n_gpu_layers = 64
    # Verbose is required to pass to the callback manager
)

AVX = 1 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [9]:
!pip install pypdf
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/GenAI.pdf")
#pages = loader.load_and_split()

In [10]:
docs = loader.load()

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [12]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
text_splitter

In [13]:
splits = text_splitter.split_documents(docs)

In [14]:
len(splits)

111

In [15]:
splits[0]

Document(page_content='See discussions, st ats, and author pr ofiles f or this public ation at : https://www .researchgate.ne t/public ation/370653602\nGenerative AI\nPreprint  · May 2023\nCITATIONS\n0READS\n20,248\n4 author s, including:\nJochen Hartmann\nTechnische Univ ersität München\n27 PUBLICA TIONS \xa0\xa0\xa0580 CITATIONS \xa0\xa0\xa0\nSEE PROFILE\nPatrick Zschech\nUniv ersity of L eipzig\n67 PUBLICA TIONS \xa0\xa0\xa01,250  CITATIONS \xa0\xa0\xa0\nSEE PROFILE\nAll c ontent f ollo wing this p age was uplo aded b y Patrick Zschech  on 05 Oct ober 2023.\nThe user has r equest ed enhanc ement of the do wnlo aded file.', metadata={'source': '/content/GenAI.pdf', 'page': 0})

In [16]:
!pip install chromadb

  Using cached urllib3-1.26.18-py2.py3-none-any.whl (143 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.1.0
    Uninstalling urllib3-2.1.0:
      Successfully uninstalled urllib3-2.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
types-requests 2.31.0.10 requires urllib3>=2, but you have urllib3 1.26.18 which is incompatible.


In [17]:
!pip install sentence-transformers

In [18]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")



In [19]:
#chroma will be the vectordb and content is our splits which is basically our chunks we created above
# converting to embeddings and storing to vectordb
#out retriever is chromadb

vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)
retriever = vectorstore.as_retriever()

In [20]:
!pip install langchainhub

  Using cached urllib3-2.1.0-py3-none-any.whl (104 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.18
    Uninstalling urllib3-1.26.18:
      Successfully uninstalled urllib3-1.26.18
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kubernetes 28.1.0 requires urllib3<2.0,>=1.24.2, but you have urllib3 2.1.0 which is incompatible.


In [21]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [22]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [23]:
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

In [24]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [25]:
rag_chain.invoke("what is generative ai")

 Generative AI is a branch of AI that can create new content such as texts, images, or audio that increasingly often cannot be distinguished anymore from human craftsmanship. It has the potential to transform domains and industries that rely on creativity, innovation, and knowledge processing.





' Generative AI is a branch of AI that can create new content such as texts, images, or audio that increasingly often cannot be distinguished anymore from human craftsmanship. It has the potential to transform domains and industries that rely on creativity, innovation, and knowledge processing.\n\n\n\n'

In [26]:
rag_chain.invoke("what is the capital of india")

Llama.generate: prefix-match hit


 The capital of India is New Delhi.

' The capital of India is New Delhi.'

In [27]:
def context_generator(query):
  retrieved_docs = retriever.get_relevant_documents(
    query
)
  context = ''

  for i in range(len(retrieved_docs)):

    context = context + str(retrieved_docs[i].page_content)
  return context

In [28]:
query = "what is the capital of india"

In [29]:
context = context_generator(query)

In [30]:
context

'et al. 2014 ). A GAN consists of two neural networks that contest with each other in the form of a zero-sum\ngame, so that samples from a speciﬁc distribution can be generated. Formally, the ﬁrst network Gis called\nthe generator, which generates candidate samples. The second network Dis called the discriminator, which\nevaluates how likely the candidate samples come from a desired distribution. Thanks to the adversarial\nlearning objective, the generator learns to map from a latent space to a data distribution of interest, while\nthe discriminator distinguishes candidates produced by the generator from the true data distribution (seeFig. 2).\n(Large) language model A (large) language model (LLM) refers to neural networks for modeling and generating text data that\ntypically combine three characteristics. First, the language model uses a large-scale, sequential neural\nnetwork (e.g., transformer with an attention mechanism). Second, the neural network is pre-trained throughdistributio

In [31]:
template = f"""Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.

Context: {context}
Question: {query}
Helpful Answer:"""

In [32]:
llm(template)

Llama.generate: prefix-match hit


 I don't know the answer to that question. The context provided does not mention India or its capital.

" I don't know the answer to that question. The context provided does not mention India or its capital."

In [33]:
template

"Use the following pieces of context to answer the question at the end.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\nUse three sentences maximum and keep the answer as concise as possible.\n\nContext: et al. 2014 ). A GAN consists of two neural networks that contest with each other in the form of a zero-sum\ngame, so that samples from a speciﬁc distribution can be generated. Formally, the ﬁrst network Gis called\nthe generator, which generates candidate samples. The second network Dis called the discriminator, which\nevaluates how likely the candidate samples come from a desired distribution. Thanks to the adversarial\nlearning objective, the generator learns to map from a latent space to a data distribution of interest, while\nthe discriminator distinguishes candidates produced by the generator from the true data distribution (seeFig. 2).\n(Large) language model A (large) language model (LLM) refers to neural networks for modeling and g

In [34]:
query = "what is generative ai"

In [35]:
context = context_generator(query)

In [36]:
context

'reﬂection of the different design activities to ensure thenecessary learning (Schoormann et al. 2023a ).\n5 Conclusion\nGenerative AI is a branch of AI that can create new content\nsuch as texts, images, or audio that increasingly often\ncannot be distinguished anymore from human craftsman-\nship. For this reason, generative AI has the potential totransform domains and industries that rely on creativity,\ninnovation, and knowledge processing. In particular, it\nenables new applications that were previously impossibleor impractical for automation, such as realistic virtual\nassistants, personalized education and service, and digital\nart. As such, generative AI has substantial implications forBISE practitioners and scholars as an interdisciplinary\nresearch community. In our Catchword article, we offered\na conceptualization of the principles of generative AI alonga model-, system-, and application-level view as well as a\nsocial-technical view and described limitations of currentdistr

In [38]:
template = f"""Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.

Context: {context}
Question: {query}
Helpful Answer:"""

In [39]:
llm(template)

Llama.generate: prefix-match hit


 Generative AI refers to artificial intelligence that can create new content such as texts, images, or audio. It has the potential to transform domains and industries that rely on creativity, innovation, and knowledge processing, such as realistic virtual assistants, personalized education and service, and digital art.

Please note that I'm just an AI and not a human, so my answer may not be as nuanced or detailed as a human expert in the field might provide.

" Generative AI refers to artificial intelligence that can create new content such as texts, images, or audio. It has the potential to transform domains and industries that rely on creativity, innovation, and knowledge processing, such as realistic virtual assistants, personalized education and service, and digital art.\n\nPlease note that I'm just an AI and not a human, so my answer may not be as nuanced or detailed as a human expert in the field might provide."